# Experimenting with the AMASS dataset

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

In [ ]:
import os.path as osp

import torch
import numpy as np

from human_body_prior.tools.omni_tools import copy2cpu as c2c

support_dir = '/home/kaseris/Documents/amass/support_data'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
amass_npz_fname = osp.join(support_dir, 'github_data/dmpl_sample.npz')
bdata = np.load(amass_npz_fname)

subject_gender = bdata['gender'].tostring().decode('utf-8')

Let's see what's inside the bdata

- What are the `dmpls`? Some body parameters????
- What are the `betas`? Body parameters

In [ ]:
print(f'The available keys from the read file are: {list(bdata.keys())}')
print(f'betas: {bdata["betas"].shape}')
print(f'dmpls: {bdata["dmpls"].shape}')
print(f'The subject gender is: {subject_gender}')

In [ ]:
from human_body_prior.body_model.body_model import BodyModel

dmpl_fname = osp.join(support_dir, f'body_models/dmpls/{subject_gender}/model.npz')
bm_fname = osp.join(support_dir, f'body_models/smplh/{subject_gender}/model.npz')

num_betas = 16 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_fname=bm_fname, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname).to(comp_device)
faces = c2c(bm.f)

The provided sample data has the original mocap marker data.

In the real AMASS dataset, we include only markers for the test set.

For the rest of the subsets you can obtain the marker data from their respective websites.

In the following we make PyTorch tensors for parameters controlling different part of the body model.

In [ ]:
time_length = len(bdata['trans'])
body_parms = {
    'root_orient': torch.Tensor(bdata['poses'][:, :3]).to(comp_device), # controls the global root orientation
    'pose_body': torch.Tensor(bdata['poses'][:, 3:66]).to(comp_device), # controls the body
    'pose_hand': torch.Tensor(bdata['poses'][:, 66:]).to(comp_device), # controls the finger articulation
    'trans': torch.Tensor(bdata['trans']).to(comp_device), # controls the global body position
    'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=time_length, axis=0)).to(comp_device), # controls the body shape. Body shape is static
    'dmpls': torch.Tensor(bdata['dmpls'][:, :num_dmpls]).to(comp_device) # controls soft tissue dynamics
}
print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in body_parms.items()])))
print('time_length = {}'.format(time_length))

In [ ]:
import trimesh
from body_visualizer.tools.vis_tools import colors
from body_visualizer.mesh.mesh_viewer import MeshViewer
from body_visualizer.mesh.sphere import points_to_spheres
from body_visualizer.tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

In [ ]:
body_pose_beta = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas']})

def vis_body_pose_beta(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose_beta.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

vis_body_pose_beta(fId=0)



Now the model has a more realistic hand posture

In [ ]:
body_pose_hand = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas', 'pose_hand']})

def vis_body_pose_hand(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose_hand.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

vis_body_pose_hand(fId=0)

Visualize the body joints

In [ ]:
def vis_body_joints(fId = 0):
    joints = c2c(body_pose_hand.Jtr[fId])
    joints_mesh = points_to_spheres(joints, point_color = colors['red'], radius=0.005)

    mv.set_static_meshes([joints_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

vis_body_joints(fId=0)

In [ ]:
body_dmpls = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas', 'pose_hand', 'dmpls']})

def vis_body_dmpls(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_dmpls.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

vis_body_dmpls(fId=0)